# Local Foundry Inference

## Step 1 - Load Environment

Uri and model id of the Foundry Local and Phi-4-reasoning model is loaded from the [configuration file](../setup/setup.ps1). 

In [4]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"
#r "nuget: DotNetEnv, 3.1.1"

using DotNetEnv;

string configurationFile = @"../config/config.env";
Env.Load(configurationFile);

string modelUri = Environment.GetEnvironmentVariable("ModelUri") ?? "ModelUri not found";
string modelId = Environment.GetEnvironmentVariable("ModelId") ?? "ModelId not found";
modelUri = modelUri + "/v1";

Console.WriteLine("Configuration loaded: ");
Console.WriteLine($"\tModel Uri: {modelUri}");
Console.WriteLine($"\tModel Id: {modelId}");

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.4 DotNetEnv, 3.1.1

Configuration loaded: 
	Model Uri: http://localhost:5273/v1
	Model Id: Phi-4-mini-reasoning-qnn-npu


## Step 2 - Create Azure OpenAI Client Instance

An Azure OpenAI Client instance is created using the previously loaded Foundry Local uri. API key credentials aren't needed as Foundry Local doesn't check Auth/AuthZ currently.


In [5]:
using OpenAI;
using OpenAI.Chat;
using System.ClientModel;

ApiKeyCredential apiKeyCredential = new ApiKeyCredential("not-needed-for-local");
OpenAIClientOptions openAIClientOptions = new OpenAIClientOptions(){
    Endpoint = new Uri(modelUri)
};
OpenAIClient openAIClient = new OpenAIClient(apiKeyCredential, openAIClientOptions);

Console.WriteLine($"OpenAIClient crated ...");

OpenAIClient crated ...


## Step 3 - Create Chat Client

A default chat client is created and the model id of the loaded SLM is provided during chat client creation. The model id differs from the model alias as it provide additional information about the processor architecture. For example the model id of the phi-4-mini-reasoning model on a Surfae Pro Copilot+ PC with Qualcomm NPU:

| Model Alias | Model Id |
| ----------- | -------- | 
| phi-4-mini-reasoning | Phi-4-mini-reasoning-qnn-npu |


In [9]:
ChatClient chatClient = openAIClient.GetChatClient(modelId);

string prompt = @"
    The Munich Flying Dolphins won the fictious Super Sports Champion Ship 2024!
    Show strategies how this could be verified.
";

ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    Temperature = 0.1f
};


## Step 4 - Chat Completion (Streaming)

Prompt and additional grounding is provided to the Foundry Local loaded model and model response is printed as it arrives.

In [10]:
CollectionResult<StreamingChatCompletionUpdate> streaminChatCompletionUpdate = 
    chatClient.CompleteChatStreaming(prompt);

Console.WriteLine($"Model repsonse: ");
foreach (StreamingChatCompletionUpdate streamingChatCompletionUpdate in streaminChatCompletionUpdate)
{
    if (streamingChatCompletionUpdate.ContentUpdate.Count > 0)
    {
        Console.Write(streamingChatCompletionUpdate.ContentUpdate[0].Text);
    }
}

Model repsonse: 
<think>
Okay, so I need to figure out how the Munich Flying Dolphins won the fictitious Super Sports Champion Ship 2024. Hmm, first, let's break down the problem. The user is asking to show how the victory could be verified. So, the key here is to outline the verification process or the steps that confirm their victory.

First, I should consider what the Super Sports Champion Ship entails. Since it's a fictitious event, the specifics can be made up, but the key is to outline a plausible verification process. The key elements are the event's name, the team involved (Munich Flying Dolphins), and the year 2024.

So, verification usually involves checking their performance in the tournament. Steps might include checking their match results, standings, possibly trophies, certificates, or official announcements.